In [93]:
import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
np.random.seed(4590)

In [94]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_hist_trans = pd.read_csv('historical_transactions.csv')
df_new_merchant_trans = pd.read_csv('new_merchant_transactions.csv')
df_merchants = pd.read_csv('merchants.csv')

In [95]:
hist_trans_merch = df_hist_trans.merge(df_merchants, on=['merchant_id', 'merchant_category_id', 'subsector_id', 'city_id', 'state_id'], how='left')
new_trans_merch = df_new_merchant_trans.merge(df_merchants, on=['merchant_id', 'merchant_category_id', 'subsector_id', 'city_id', 'state_id'], how='left')

In [96]:
hist_trans_merch.head()

,authorized_flag,card_id,city_id,category_1_x,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_2_y
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,...,1.082451,3.0,1.14,1.114135,6.0,1.19,1.156844,12.0,Y,1.0
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,...,1.052071,3.0,1.06,1.058605,6.0,1.05,1.062087,12.0,Y,1.0
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,...,0.974653,3.0,0.98,0.967058,6.0,0.97,0.956668,12.0,Y,1.0
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,...,1.053443,3.0,0.88,0.897406,6.0,0.86,0.864394,12.0,Y,1.0
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,...,1.082451,3.0,1.14,1.114135,6.0,1.19,1.156844,12.0,Y,1.0


In [97]:
def checking_missing_value(data):
    print(data.isna().sum() / data.shape[0])
    
checking_missing_value(hist_trans_merch)

authorized_flag                0.000000
card_id                        0.000000
city_id                        0.000000
category_1_x                   0.000000
installments                   0.000000
category_3                     0.006103
merchant_category_id           0.000000
merchant_id                    0.004741
month_lag                      0.000000
purchase_amount                0.000000
purchase_date                  0.000000
category_2_x                   0.091984
state_id                       0.000000
subsector_id                   0.000000
merchant_group_id              0.349832
numerical_1                    0.349832
numerical_2                    0.349832
category_1_y                   0.349832
most_recent_sales_range        0.349832
most_recent_purchases_range    0.349832
avg_sales_lag3                 0.350141
avg_purchases_lag3             0.349832
active_months_lag3             0.349832
avg_sales_lag6                 0.350141
avg_purchases_lag6             0.349832


In [98]:
%%time

mode_cols = ['category_3', 'merchant_id', 'category_2_x', 'merchant_group_id', 'category_1_y', 'most_recent_sales_range'
            , 'most_recent_purchases_range', 'active_months_lag3', 'active_months_lag6', 'active_months_lag12', 'category_4'
            , 'category_2_y']
mean_cols = ['numerical_1', 'numerical_2', 'avg_sales_lag3', 'avg_purchases_lag3', 'avg_sales_lag6', 'avg_purchases_lag6', 'avg_sales_lag12', 'avg_purchases_lag12' ]

for df in [hist_trans_merch, new_trans_merch]:
    for col in mode_cols:
        df[col].fillna(df[col].mode(), inplace=True)
        
    for col in mean_cols:
        df[col].fillna(df[col].mean(), inplace=True)

CPU times: user 36.7 s, sys: 9.21 s, total: 45.9 s
Wall time: 47.8 s


In [99]:
checking_missing_value(hist_trans_merch)

authorized_flag                0.000000
card_id                        0.000000
city_id                        0.000000
category_1_x                   0.000000
installments                   0.000000
category_3                     0.006103
merchant_category_id           0.000000
merchant_id                    0.004741
month_lag                      0.000000
purchase_amount                0.000000
purchase_date                  0.000000
category_2_x                   0.091984
state_id                       0.000000
subsector_id                   0.000000
merchant_group_id              0.349832
numerical_1                    0.000000
numerical_2                    0.000000
category_1_y                   0.349832
most_recent_sales_range        0.349832
most_recent_purchases_range    0.349832
avg_sales_lag3                 0.000000
avg_purchases_lag3             0.000000
active_months_lag3             0.349832
avg_sales_lag6                 0.000000
avg_purchases_lag6             0.000000


In [100]:
%%time

for df in [hist_trans_merch, new_trans_merch]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['year'] = df['purchase_date'].dt.year
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1_x'] = df['category_1_x'].map({'Y':1, 'N':0}) 
    #https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/73244
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    
    #features from merchant.csv
    df['category_1_y'] = df['category_1_y'].map({'Y':1, 'N':0})     
    df['most_recent_sales_range'] = df['most_recent_sales_range'].map({'A':0, 'B':0, 'C':0, 'D':0, 'E':0})         
    df['most_recent_purchases_range'] = df['most_recent_purchases_range'].map({'A':0, 'B':0, 'C':0, 'D':0, 'E':0})             
    df['category_4'] = df['category_4'].map({'Y':1, 'N':0})         
    

CPU times: user 35.1 s, sys: 12.9 s, total: 48 s
Wall time: 52 s


In [101]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [102]:
%%time

aggs = {}
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id']:
    aggs[col] = ['nunique']

aggs['purchase_amount'] = ['sum','max','min','mean','var']
aggs['installments'] = ['sum','max','min','mean','var']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ['mean']
aggs['authorized_flag'] = ['sum', 'mean']
aggs['weekend'] = ['sum', 'mean']
aggs['category_1_x'] = ['sum', 'mean']
aggs['card_id'] = ['size']

# features from merchant
aggs['numerical_1'] = ['sum','max','min','mean','var']
aggs['numerical_1'] = ['sum','max','min','mean','var']
aggs['category_1_y'] = ['mean','var']
aggs['most_recent_sales_range'] = ['mean','var']
aggs['most_recent_purchases_range'] = ['mean','var']
aggs['avg_sales_lag3'] = ['sum','max','min','mean','var']
for s in ['3', '6', '12']:    
    aggs['avg_purchases_lag' + s] = ['sum','max','min','mean','var']
    aggs['active_months_lag' + s] = ['sum','max','min','mean','var']
    aggs['avg_sales_lag' + s] = ['sum','max','min','mean','var']

aggs['category_4'] = ['mean','var']
aggs['category_2_y'] = ['mean','var']


for col in ['category_2_x','category_3']:
    hist_trans_merch[col+'_mean'] = hist_trans_merch.groupby([col])['purchase_amount'].transform('mean')
    aggs[col+'_mean'] = ['mean']    

new_columns = get_new_columns('hist',aggs)
df_hist_trans_group = hist_trans_merch.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['hist_purchase_date_diff'] = (df_hist_trans_group['hist_purchase_date_max'] - df_hist_trans_group['hist_purchase_date_min']).dt.days
df_hist_trans_group['hist_purchase_date_average'] = df_hist_trans_group['hist_purchase_date_diff']/df_hist_trans_group['hist_card_id_size']
df_hist_trans_group['hist_purchase_date_uptonow'] = (datetime.datetime.today() - df_hist_trans_group['hist_purchase_date_max']).dt.days
df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group;gc.collect()

CPU times: user 1min 42s, sys: 1min 28s, total: 3min 10s
Wall time: 3min 56s


In [103]:
%%time

aggs = {}
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id']:
    aggs[col] = ['nunique']
    aggs['purchase_amount'] = ['sum','max','min','mean','var']
    aggs['installments'] = ['sum','max','min','mean','var']
    aggs['purchase_date'] = ['max','min']
    aggs['month_lag'] = ['max','min','mean','var']
    aggs['month_diff'] = ['mean']
    aggs['weekend'] = ['sum', 'mean']
    aggs['category_1_x'] = ['sum', 'mean']
    aggs['card_id'] = ['size']

# features from merchant
aggs['numerical_1'] = ['sum','max','min','mean','var']
aggs['numerical_1'] = ['sum','max','min','mean','var']
aggs['category_1_y'] = ['mean','var']
aggs['most_recent_sales_range'] = ['mean','var']
aggs['most_recent_purchases_range'] = ['mean','var']
aggs['avg_sales_lag3'] = ['sum','max','min','mean','var']
for s in ['3', '6', '12']:    
    aggs['avg_purchases_lag' + s] = ['sum','max','min','mean','var']
    aggs['active_months_lag' + s] = ['sum','max','min','mean','var']
    aggs['avg_sales_lag' + s] = ['sum','max','min','mean','var']

aggs['category_4'] = ['mean','var']
aggs['category_2_y'] = ['mean','var']    
    
for col in ['category_2_x','category_3']:
    new_trans_merch[col+'_mean'] = new_trans_merch.groupby([col])['purchase_amount'].transform('mean')
    aggs[col+'_mean'] = ['mean']
    
new_columns = get_new_columns('new_hist',aggs)
df_hist_trans_group = new_trans_merch.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['new_hist_purchase_date_diff'] = (df_hist_trans_group['new_hist_purchase_date_max'] - df_hist_trans_group['new_hist_purchase_date_min']).dt.days
df_hist_trans_group['new_hist_purchase_date_average'] = df_hist_trans_group['new_hist_purchase_date_diff']/df_hist_trans_group['new_hist_card_id_size']
df_hist_trans_group['new_hist_purchase_date_uptonow'] = (datetime.datetime.today() - df_hist_trans_group['new_hist_purchase_date_max']).dt.days
df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group;gc.collect()

CPU times: user 13.4 s, sys: 4.06 s, total: 17.5 s
Wall time: 21.2 s


In [104]:
del hist_trans_merch;gc.collect()
del new_trans_merch;gc.collect()
df_train.head(5)

,first_active_month,card_id,feature_1,feature_2,feature_3,target,hist_month_nunique,hist_hour_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,...,new_hist_avg_sales_lag12_var,new_hist_category_4_mean,new_hist_category_4_var,new_hist_category_2_y_mean,new_hist_category_2_y_var,new_hist_category_2_x_mean_mean,new_hist_category_3_mean_mean,new_hist_purchase_date_diff,new_hist_purchase_date_average,new_hist_purchase_date_uptonow
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,9,23,35,7,...,27.551899,1.0,0.0,1.00,0.00,-0.568720,-0.630986,54.0,2.347826,271.0
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,12,24,50,7,...,0.044347,1.0,0.0,1.00,0.00,-0.568720,-0.606311,56.0,9.333333,301.0
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,10,14,22,7,...,NaN,1.0,NaN,5.00,NaN,-0.549015,-0.630986,0.0,0.000000,272.0
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,6,16,20,7,...,0.078581,1.0,0.0,4.00,0.00,-0.558049,-0.606311,41.0,5.857143,282.0
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,4,22,17,7,...,729.972348,1.0,0.0,3.85,0.45,-0.557196,-0.587556,57.0,1.583333,272.0


In [105]:
df_train['outliers'] = 0
df_train.loc[df_train['target'] < -30, 'outliers'] = 1
df_train['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

In [106]:
%%time

for df in [df_train,df_test]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_hist_first_buy'] = (df['new_hist_purchase_date_min'] - df['first_active_month']).dt.days
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_hist_purchase_date_max',\
                     'new_hist_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9
    df['card_id_total'] = df['new_hist_card_id_size']+df['hist_card_id_size']
    df['purchase_amount_total'] = df['new_hist_purchase_amount_sum']+df['hist_purchase_amount_sum']

for f in ['feature_1','feature_2','feature_3']:
    order_label = df_train.groupby([f])['outliers'].mean()
    df_train[f] = df_train[f].map(order_label)
    df_test[f] = df_test[f].map(order_label)

CPU times: user 730 ms, sys: 520 ms, total: 1.25 s
Wall time: 1.26 s


In [107]:
df_train_columns = [c for c in df_train.columns if c not in ['card_id', 'first_active_month','target','outliers']]

target = df_train['target']
del df_train['target']

In [108]:
#remove insignificant features measured by XGboost

df_train_columns = [c for c in df_train.columns if c not in ['card_id', 'first_active_month','target','outliers',
                                                             'new_hist_active_months_lag3_max',
                    'hist_most_recent_sales_range_var',
                    'hist_most_recent_sales_range_mean',
                    'hist_most_recent_purchases_range_var',
                    'hist_most_recent_purchases_range_mean',
                    'new_hist_most_recent_sales_range_var',
                    'new_hist_most_recent_sales_range_mean',
                    'hist_active_months_lag6_max',
                    'new_hist_most_recent_purchases_range_mean',
                    'hist_active_months_lag3_max', 'new_hist_active_months_lag6_max',
                    'new_hist_most_recent_purchases_range_var',
                    'new_hist_active_months_lag6_var',
                    'new_hist_active_months_lag3_min',
                    'new_hist_active_months_lag6_min', 'hist_active_months_lag3_min',
                    'new_hist_active_months_lag3_mean', 'hist_active_months_lag3_var',
                    'hist_avg_purchases_lag6_sum', 'hist_avg_purchases_lag3_max',
                    'new_hist_active_months_lag6_sum', 'hist_avg_purchases_lag3_mean',
                    'hist_avg_purchases_lag3_sum', 'new_hist_active_months_lag12_max',
                    'hist_avg_purchases_lag6_max', 'hist_active_months_lag3_mean',
                    'hist_avg_purchases_lag6_mean', 'hist_active_months_lag12_max',
                    'hist_avg_purchases_lag12_sum', 'new_hist_month_nunique',
                    'hist_avg_purchases_lag12_max', 'hist_avg_purchases_lag12_mean',
                    'new_hist_active_months_lag12_min',
                    'new_hist_active_months_lag3_sum', 'hist_active_months_lag6_min',
                    'hist_year_nunique', 'new_hist_active_months_lag3_var',
                    'new_hist_active_months_lag6_mean', 'hist_avg_sales_lag6_max',
                    'hist_avg_sales_lag3_max', 'new_hist_category_2_y_var',
                    'new_hist_month_lag_min', 'new_hist_weekend_sum',
                    'new_hist_active_months_lag12_var']]

In [109]:
%%time

param = {'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 4,
         "random_state": 4590}
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=4590)
oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train,df_train['outliers'].values)):
    print("fold {}".format(fold_))
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][df_train_columns], label=target.iloc[trn_idx])#, categorical_feature=categorical_feats)
    val_data = lgb.Dataset(df_train.iloc[val_idx][df_train_columns], label=target.iloc[val_idx])#, categorical_feature=categorical_feats)
    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 100)
    oof[val_idx] = clf.predict(df_train.iloc[val_idx][df_train_columns], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = df_train_columns
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(df_test[df_train_columns], num_iteration=clf.best_iteration) / folds.n_splits

np.sqrt(mean_squared_error(oof, target))

fold 0
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 3.66078	valid_1's rmse: 3.72394
[200]	training's rmse: 3.5797	valid_1's rmse: 3.69193
[300]	training's rmse: 3.5302	valid_1's rmse: 3.67766
[400]	training's rmse: 3.49211	valid_1's rmse: 3.66957
[500]	training's rmse: 3.46102	valid_1's rmse: 3.66443
[600]	training's rmse: 3.43395	valid_1's rmse: 3.66089
[700]	training's rmse: 3.41068	valid_1's rmse: 3.65914
[800]	training's rmse: 3.38885	valid_1's rmse: 3.65759
[900]	training's rmse: 3.36799	valid_1's rmse: 3.65604
[1000]	training's rmse: 3.3494	valid_1's rmse: 3.65548
[1100]	training's rmse: 3.33178	valid_1's rmse: 3.65495
[1200]	training's rmse: 3.3144	valid_1's rmse: 3.65462
[1300]	training's rmse: 3.29833	valid_1's rmse: 3.65458
[1400]	training's rmse: 3.28275	valid_1's rmse: 3.65484
Early stopping, best iteration is:
[1309]	training's rmse: 3.29674	valid_1's rmse: 3.65444
fold 1
Training until validation scores don't improve for 100 rounds

In [114]:
np.sqrt(mean_squared_error(oof, target))

3.655902844781449

In [110]:
cols = (feature_importance_df[["Feature", "importance"]]
        .groupby("Feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="Feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.savefig('./output/lgbm_importances.png')

In [111]:
sub_df = pd.DataFrame({"card_id":df_test["card_id"].values})
sub_df["target"] = predictions
sub_df.to_csv("./output/submission_4.csv", index=False)

In [112]:
# class StackingModel():
#     def __init__():
#         pass
    
#     def fit():
#         pass
    
#     def predict():
#         pass
    

In [113]:
# sorted_features = feature_importance_df[['Feature', 'importance']].groupby('Feature').mean().sort_values(by='importance')
# sorted_features[sorted_features['importance'] < 30].index.values

# ['new_hist_active_months_lag3_max',
#        'hist_most_recent_sales_range_var',
#        'hist_most_recent_sales_range_mean',
#        'hist_most_recent_purchases_range_var',
#        'hist_most_recent_purchases_range_mean',
#        'new_hist_most_recent_sales_range_var',
#        'new_hist_most_recent_sales_range_mean',
#        'hist_active_months_lag6_max',
#        'new_hist_most_recent_purchases_range_mean',
#        'hist_active_months_lag3_max', 'new_hist_active_months_lag6_max',
#        'new_hist_most_recent_purchases_range_var',
#        'new_hist_active_months_lag6_var',
#        'new_hist_active_months_lag3_min',
#        'new_hist_active_months_lag6_min', 'hist_active_months_lag3_min',
#        'new_hist_active_months_lag3_mean', 'hist_active_months_lag3_var',
#        'hist_avg_purchases_lag6_sum', 'hist_avg_purchases_lag3_max',
#        'new_hist_active_months_lag6_sum', 'hist_avg_purchases_lag3_mean',
#        'hist_avg_purchases_lag3_sum', 'new_hist_active_months_lag12_max',
#        'hist_avg_purchases_lag6_max', 'hist_active_months_lag3_mean',
#        'hist_avg_purchases_lag6_mean', 'hist_active_months_lag12_max',
#        'hist_avg_purchases_lag12_sum', 'new_hist_month_nunique',
#        'hist_avg_purchases_lag12_max', 'hist_avg_purchases_lag12_mean',
#        'new_hist_active_months_lag12_min',
#        'new_hist_active_months_lag3_sum', 'hist_active_months_lag6_min',
#        'hist_year_nunique', 'new_hist_active_months_lag3_var',
#        'new_hist_active_months_lag6_mean', 'hist_avg_sales_lag6_max',
#        'hist_avg_sales_lag3_max', 'new_hist_category_2_y_var',
#        'new_hist_month_lag_min', 'new_hist_weekend_sum',
#        'new_hist_active_months_lag12_var']